In [2]:
import torch
from utils.profiler import profile

from upfirdn2d.new.upfirdn2d import upfirdn2d as new
from upfirdn2d.original.upfirdn2d import upfirdn2d as original


Using /home/user/.cache/torch_extensions/py38_cu115 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/user/.cache/torch_extensions/py38_cu115/upfirdn2d_new/build.ninja...
Building extension module upfirdn2d_new...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
[1/3] c++ -MMD -MF upfirdn2d.o.d -DTORCH_EXTENSION_NAME=upfirdn2d_new -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1013\" -isystem /home/user/.conda/envs/user/lib/python3.8/site-packages/torch/include -isystem /home/user/.conda/envs/user/lib/python3.8/site-packages/torch/include/torch/csrc/api/include -isystem /home/user/.conda/envs/user/lib/python3.8/site-packages/torch/include/TH -isystem /home/user/.conda/envs/user/lib/python3.8/site-packages/torch/include/THC -isystem /usr/local/cuda/include -isystem /home/user/.conda/envs/user/

In [ ]:
# Create random FP image
random_img = torch.rand((8, 1, 1024, 1024), dtype=torch.float32, device="cuda")

# Create kernel
k = torch.tensor([1, 3, 3, 1], dtype=torch.float32, device="cuda")
k = k[None, :] * k[:, None]
k /= k.sum()


# Compare output from original and modified kernels
def fn(i: int) -> None:
    original_result = original(random_img, k, up=2, down=1, pad=(0, 0))
    new_result = new(random_img, k, up=2, down=1, pad=(0, 0))
    assert new_result.allclose(original_result), f"Assertion failed at iteration {i}"


profile(fn, ["upfirdn2d"], iters=5000)